In [ ]:
from fast import *
init_printing()
use_unicode=True; use_unicode=False

In [ ]:
from sympy import sin,cos,exp,sqrt,pi,zeros,I
from numpy import array

We define the number of states and of radiation fields.

In [ ]:
Ne=3
Nl=2

We define the variables related to the laser field.

In [ ]:
E0,omega_laser=define_laser_variables(Nl)
pprint(E0,use_unicode=use_unicode)

In [ ]:
pprint(omega_laser,use_unicode=use_unicode)

We define a few important symbols.

In [ ]:
t,hbar,e=symbols("t hbar e",positive=True)
pprint([t,hbar,e],use_unicode=use_unicode)

We write two electric fields propagating trough the $\hat{x}$ direction polarized in the $\hat{z}$ direction. First the wave vectors:

In [ ]:
phi=0; theta=pi/2; alpha=pi/2; beta=0

k1=Matrix([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])
k2=Matrix([cos(phi)*sin(theta),sin(phi)*sin(theta),cos(theta)])

pprint([k1,k2],use_unicode=use_unicode)

The polarization vectors.

In [ ]:
ep1=polarization_vector(phi,theta,alpha,beta, 1)
ep2=polarization_vector(phi,theta,alpha,beta, 1)

em1=ep1.conjugate()
em2=ep2.conjugate()

ep=[ep1,ep2]
em=[em1,em2]

pprint([ep,em],use_unicode=use_unicode)

The electric field (evaluated in $\vec{R}=0$).

In [ ]:
zero_vect=Matrix([0,0,0])
E_cartesian = [(E0[l]*ep[l]*exp(-I*omega_laser[l]*t) + E0[l].conjugate()*em[l]*exp( I*omega_laser[l]*t))/2 
                    for l in range(Nl)]

pprint(E_cartesian,use_unicode=use_unicode)

We write the electric field in the helicity basis.

In [ ]:
E=[cartesian_to_helicity(E_cartesian[l]) for l in range(Nl)]
pprint(E,use_unicode=use_unicode)

We define the position operator.

In [ ]:
r=define_r_components(Ne,helicity=True,explicitly_hermitian=True)
#Ladder means that r_{p;31}=0
r=[ri.subs({r[0][1,0]:0,r[1][1,0]:0,r[2][1,0]:0}) for ri in r]
pprint(r,num_columns=150,use_unicode=use_unicode)

The frequencies of the energy levels, the resonant frequencies, and the decay frequencies.

In [ ]:
omega_level,omega,gamma=define_frequencies(Ne,explicitly_antisymmetric=True)
#Ladder means gamma31=0
gamma=gamma.subs({gamma[1,0]:0})

pprint(omega_level,use_unicode=use_unicode)

In [ ]:
pprint(omega,use_unicode=use_unicode)

In [ ]:
pprint(gamma,use_unicode=use_unicode)

The atomic hamiltonian is

In [ ]:
H0=Matrix([[hbar*omega_level[i]*KroneckerDelta(i,j) for j in range(Ne)] for i in range(Ne)])
pprint(H0,use_unicode=use_unicode)

The interaction hamiltonian is

In [ ]:
zero_matrix=zeros(Ne,Ne)
H1=sum([ e*helicity_dot_product(E[l],r) for l in range(Nl)],zero_matrix)

and the complete hamiltonian is

In [ ]:
H=H0+H1

# Rotating wave approximation
Notice that the electric field can be separated by terms with positive and negative frequency:

In [ ]:
E_cartesian_p=[E0[l]/2*ep[l]*exp(-I*omega_laser[l]*t) for l in range(Nl)]
E_cartesian_m=[E0[l].conjugate()/2*em[l]*exp( I*omega_laser[l]*t) for l in range(Nl)]

E_p=[cartesian_to_helicity(E_cartesian_p[l]) for l in range(Nl)]
E_m=[cartesian_to_helicity(E_cartesian_m[l]) for l in range(Nl)]

pprint([E_p,E_m],use_unicode=use_unicode)

In [ ]:
pprint( simplify(sum([E[l] for l in range(Nl)],zero_vect)-(sum([E_p[l]+E_m[l] for l in range(Nl)],zero_vect) )),use_unicode=use_unicode)

The position operator can also be separated in this way. We go to the interaction picture (with $\hat{H}_0$ as the undisturbed hamiltonian)

In [ ]:
r_I=[ Matrix([[exp(I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)] for i in range(Ne)]) for p in range(3)]
pprint(r_I[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I[2],use_unicode=use_unicode)

Which can be decomposed in positive and negative frequencies as

In [ ]:
r_I_p=[ Matrix([[ delta_greater(j,i)*exp(-I*omega[j,i]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_p[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I_p[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I_p[2],use_unicode=use_unicode)

In [ ]:
r_I_m=[ Matrix([[ delta_lesser( j,i)*exp( I*omega[i,j]*t)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_I_m[0],use_unicode=use_unicode)

In [ ]:
pprint(r_I_m[1],use_unicode=use_unicode)

In [ ]:
pprint(r_I_m[2],use_unicode=use_unicode)

that summed equal $\vec{\hat{r}}_I$

In [ ]:
pprint( [r_I[p]-(r_I_p[p]+r_I_m[p]) for p in range(3)] ,use_unicode=use_unicode)

Thus the interaction hamiltonian in the interaciton picture is
\begin{equation}
    \hat{H}_{1I}=e\vec{E}\cdot \vec{\hat{r}}_I= e(\vec{E}^{(+)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(-)}_I)
\end{equation}

In [ ]:
H1I=sum([ e*helicity_dot_product(E[l],r_I) for l in range(Nl)],zero_matrix)

Since both $\omega^l$ and $\omega_{ij}$ are in the order of THz, the terms that have frequencies with the same sign are summed, and thus also of the order of THz. The frequencies in the terms with oposite signs however, are detunings of the order of MHz. Since we are only interested in the coarse-grained evolution of the density matrix, we may omit the fast terms and approximate

\begin{equation}
    \hat{H}_{1I} \simeq \hat{H}_{1I,RWA}= e( \vec{E}^{(+)}\cdot \vec{\hat{r}}^{(-)}_I + \vec{E}^{(-)}\cdot \vec{\hat{r}}^{(+)}_I )
\end{equation}

That is known as the rotating wave approximation (RWA).

In [ ]:
H1IRWA=sum( [ (e*(helicity_dot_product(E_p[l],r_I_m)+helicity_dot_product(E_m[l],r_I_p))) for l in range(Nl)],zero_matrix)

For instance, the $(\hat{H}_{1I,RWA})_{21}$ element is

In [ ]:
pprint(H1IRWA[2,0].expand(),use_unicode=use_unicode)

But if the detuning $\omega_{31}-\omega^1 \ll \omega_{31}-\omega^2$ (the second field is far detuned from the $1 \rightarrow 3$ transition), then $\omega_{31}-\omega^2$ may be also considered too high a frequency to be relevant to coarse-grained evolution. So we might neclect that term in $(\hat{H}_{1I,RWA})_{31}$ and similarly neglect the $\omega_{32}-\omega^1$ for term in $(\hat{H}_{1I,RWA})_{32}$:

In [ ]:
pprint(H1IRWA[2,1].expand(),use_unicode=use_unicode)

In other words, if the detunings in our experiments allow the approximmation, we might choose which frequency components $\omega^l$ excite which transitions. Let us say that $L_{ij}$ is the set of $l$ such that $\omega^l$ excites the transition $i\rightarrow j$

In [ ]:
Lij=[[1,3,[1]],[2,3,[2]]]
Lij=formatLij(Lij,Ne)
print array(Lij)

Thus the interacion hamiltonian in the interaction picture can be approximated as

In [ ]:
H1IRWA =sum([ e*( helicity_dot_product( E_p[l],vector_element(r_I_m,i,j)) ) * ket(i+1,Ne)*bra(j+1,Ne) 
            for l in range(Nl) for j in range(Ne) for i in range(Ne) if l+1 in Lij[i][j] ],zero_matrix)
H1IRWA+=sum([ e*( helicity_dot_product( E_m[l],vector_element(r_I_p,i,j)) ) * ket(i+1,Ne)*bra(j+1,Ne) 
            for l in range(Nl) for j in range(Ne) for i in range(Ne) if l+1 in Lij[i][j] ],zero_matrix)

pprint(H1IRWA,num_columns=150,use_unicode=use_unicode)

Returning to the Schrödinger picture we have.

In [ ]:
r_p=[ Matrix([[ delta_greater(j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_p,num_columns=150,use_unicode=use_unicode)

In [ ]:
r_m=[ Matrix([[ delta_lesser( j,i)*r[p][i,j] for j in range(Ne)]for i in range(Ne)]) for p in range(3)]
pprint(r_m,num_columns=150,use_unicode=use_unicode)

In [ ]:
pprint( [r[p]-(r_p[p]+r_m[p]) for p in range(3)] ,use_unicode=use_unicode)

Thus the interaction hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [ ]:
H1RWA =sum([ e*( helicity_dot_product( E_p[l],vector_element(r_m,i,j)) ) * ket(i+1,Ne)*bra(j+1,Ne) 
            for l in range(Nl) for j in range(Ne) for i in range(Ne) if l+1 in Lij[i][j] ],zero_matrix)
H1RWA+=sum([ e*( helicity_dot_product( E_m[l],vector_element(r_p,i,j)) ) * ket(i+1,Ne)*bra(j+1,Ne) 
            for l in range(Nl) for j in range(Ne) for i in range(Ne) if l+1 in Lij[i][j] ],zero_matrix)

pprint(H1RWA,num_columns=150,use_unicode=use_unicode)

And the complete hamiltonian in the Schrödinger picture in the rotating wave approximation is

In [ ]:
HRWA=H0+H1RWA
pprint(HRWA,use_unicode=use_unicode)

# Rotating Frame
Next we will make a phase transformation in order to eliminate the explicit time dependance of the equations.

In [ ]:
c,ctilde,phase=define_psi_coefficients(Ne)
pprint([c,ctilde,phase],use_unicode=use_unicode)

In [ ]:
psi=Matrix([ exp(I*phase[i]*t)*ctilde[i] for i in range(Ne)])
pprint(psi,use_unicode=use_unicode)

The Schrödinger equation $i\hbar \partial_t |\psi\rangle=\hat{H}_{RWA}$ is

In [ ]:
lhs=Matrix([(I*hbar*Derivative(psi[i],t).doit()).expand() for i in range(Ne)])
pprint(lhs,use_unicode=use_unicode)

In [ ]:
rhs=HRWA*psi

We multiply each of these equations by $e^{-i \theta_i t}$ and substracting $i \theta_i \tilde{c}_i$

In [ ]:
lhs_new=Matrix([simplify(  lhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(lhs_new,use_unicode=use_unicode)

In [ ]:
rhs_new=Matrix([simplify(  rhs[i]*exp(-I*phase[i]*t) +hbar*phase[i]*ctilde[i] ) for i in range(Ne)])
pprint(rhs_new,num_columns=120,use_unicode=use_unicode)

It can be seen that the equations loose their explicit time dependance only if $\omega^{1} - \theta_{1} + \theta_{3}=0$ and $\omega^{2} - \theta_{2} + \theta_{3}=0$. Which is satisfied if

In [ ]:
phase_transformation=solve([omega_laser[0]-phase[0]+phase[2],omega_laser[1]-phase[1]+phase[2]],[phase[1],phase[2]],
                           dict=True)[0]
pprint(phase_transformation,use_unicode=use_unicode)

There is a free parameter $\theta_1$, which is to be expected, since state vetors $|\psi\rangle$ always have a global phase invariance

In [ ]:
pprint(psi.subs(phase_transformation),use_unicode=use_unicode)

Thus the equations become

In [ ]:
pprint(lhs_new,use_unicode=use_unicode)

In [ ]:
rhs_new=simplify(rhs_new.subs(phase_transformation)).expand()
pprint(rhs_new,use_unicode=use_unicode)

It can be seen that this is the Schrödinger equation derived from an effective hamiltonian $\tilde{H}$

In [ ]:
Htilde=Matrix([ [Derivative(rhs_new[i],ctilde[j]).doit() for j in range(Ne)] for i in range(Ne)])
pprint(Htilde,use_unicode=use_unicode)

We can see that it is convenient to choose $\theta_1=-\omega_1$ to simplify the hamiltonian. Also, we can recognize $\omega^1-\omega_3+\omega_1=\delta^1$ as the detuning of the first field relative to the atomic transition $\omega_{21}=\omega_2-\omega_1$, and the same for $\omega^2-\omega_3+\omega_2=\delta^2$.

In [ ]:
delta1,delta2=symbols("delta1 delta2",real=True)
Htilde=Htilde.subs({phase[0]:-omega_level[0]})
Htilde=Htilde.subs({omega_laser[0]:delta1+omega_level[2]-omega_level[0]})
Htilde=Htilde.subs({omega_laser[1]:delta2+omega_level[2]-omega_level[1]})

Htilde=Htilde.expand()

pprint(Htilde,use_unicode=use_unicode)

If we define the Rabi frequencies $\Omega_1 =e E_0^1 r_{0;21}/\hbar$ and $\Omega_2 =e E_0^2 r_{0;32}/\hbar$

In [ ]:
Omega1,Omega2=symbols("Omega1 Omega2",real=True)
Omega1,Omega2=symbols("Omega1 Omega2")
Htilde=Htilde.subs({E0[0]:Omega1*hbar/r[1][2,0]/e})
Htilde=Htilde.subs({E0[1]:Omega2*hbar/r[1][2,1]/e})

pprint(Htilde,use_unicode=use_unicode)

We define the density matrix.

In [ ]:
rho=define_density_matrix(Ne)
pprint( rho ,use_unicode=use_unicode)

The hamiltonian part of the equations is
\begin{equation}
    \dot{\hat{\rho}}=\frac{i}{\hbar}[\hat{\rho}, \hat{\tilde{H}}]
\end{equation}

In [ ]:
hamiltonian_terms=(I/hbar*(rho*Htilde-Htilde*rho)).expand()

There are two Lindblad operators, since there are two spontaneous decay channels.

In [ ]:
lindblad_terms =gamma[2,0]*lindblad_operator(ket(1,Ne)*bra(3,Ne),rho)
lindblad_terms+=gamma[2,1]*lindblad_operator(ket(2,Ne)*bra(3,Ne),rho)

pprint(lindblad_terms, num_columns=120,use_unicode=use_unicode)

# Optical Bloch Equations
The Optical Bloch equations are thus.

In [ ]:
eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120,use_unicode=use_unicode)

which is how most literature will show the equations. However, a more convenient way to express this equations is to explicitly asume a normalized and hermitian density matrix

In [ ]:
rho=define_density_matrix(Ne,explicitly_hermitian=True,normalized=True)
pprint( rho ,use_unicode=use_unicode)

In [ ]:
hamiltonian_terms = (I/hbar*(rho*Htilde-Htilde*rho)).expand()
lindblad_terms    =gamma[1,0]*lindblad_operator(ket(1,Ne)*bra(2,Ne),rho)
lindblad_terms   +=gamma[2,1]*lindblad_operator(ket(2,Ne)*bra(3,Ne),rho)

eqs=hamiltonian_terms + lindblad_terms
pprint(eqs,num_columns=120,use_unicode=use_unicode)

and only consider the equations for the populations $\rho_{ii}$ for $i>1$ and the real and imaginary parts of the coherences below the diagonal.

In [ ]:
pprint( re(eqs[1,1]) ,use_unicode=use_unicode)

In [ ]:
pprint( re(eqs[1,0]) ,use_unicode=use_unicode)

In [ ]:
pprint( im(eqs[1,0]) ,use_unicode=use_unicode)

If the density matrix is represented as a vector whose components are the these independent components of the density matrix

In [ ]:
rho_vect=define_rho_vector(rho,Ne)
pprint(rho_vect,use_unicode=use_unicode)

Then the equations can be re-written as linear combinations of these components plus an independent term.
\begin{equation}
    \dot{\vec{\rho}} = \hat{A} \vec{\rho} + \vec{b}
\end{equation}
with $\hat{A}$ a linear operator acting in this vector space and $\vec{b}$ the vector of independent terms.

In [ ]:
A,b=calculate_A_b(eqs,rho,Ne)
pprint([A,b],num_columns=120,use_unicode=use_unicode)

Explicitly, this is

In [ ]:
eqs_new=A*rho_vect - b
pprint(eqs_new,num_columns=120,use_unicode=use_unicode)

Which is the same as the equations in the previous form.

In [ ]:
ss_comp={ rho[i,j]:re(rho[i,j])+I*im(rho[i,j]) for j in range(Ne) for i in range(Ne)}

test= (eqs_new - Matrix([re(eqs[1,1]),re(eqs[2,2]),
                          re(eqs[1,0]),re(eqs[2,0]),re(eqs[2,1]),
                          im(eqs[1,0]),im(eqs[2,0]),im(eqs[2,1])])).expand()

test=test.subs(ss_comp)
pprint(test,use_unicode=use_unicode)

[1]  H.J. Metcalf and P. van der Straten. Laser Cooling and Trapping. Graduate Texts in Contempo-
rary Physics. Springer New York, 2001.

[2] Daniel Adam Steck. Quantum and Atom Optics. Oregon Center for Optics and Department of Physics, University of Oregon Copyright © 200